<a href="https://colab.research.google.com/github/Asadityasingh/DLP/blob/main/Final_LandDetect_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset
dataset = load_dataset("facebook/voxpopuli","en",split='train',streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

voxpopuli.py:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

The repository for facebook/voxpopuli contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/voxpopuli.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [3]:
subset = []
for i,example in enumerate(dataset):
  if i >= 50:
    break
  subset.append(example)

In [4]:
dataset_es = load_dataset("facebook/voxpopuli","es",split='train',streaming=True)

In [5]:
for i, example in enumerate(dataset_es):
  if i >= 50:
    break
  subset.append(example)

In [6]:
dataset_ro = load_dataset("facebook/voxpopuli","ro",split='train',streaming=True)


In [7]:
for i, example in enumerate(dataset_ro):
  if i>=50:
    break
  subset.append(example)

In [8]:
len(subset)

150

In [9]:
subset[0]

{'audio_id': '20180418-0900-PLENARY-3-en_20180418-08:50:36_17',
 'language': 0,
 'audio': {'path': 'train_part_0/20180418-0900-PLENARY-3-en_20180418-08:50:36_17.wav',
  'array': array([-0.00030518,  0.00119019,  0.00506592, ..., -0.00036621,
         -0.00027466, -0.00018311]),
  'sampling_rate': 16000},
 'raw_text': 'If you do not address this problem, the ground is there for populist nationalist forces to go on growing all over Europe.',
 'normalized_text': 'if you do not address this problem the ground is there for populist nationalist forces to go on growing all over europe.',
 'gender': 'female',
 'speaker_id': '124737',
 'is_gold_transcript': True,
 'accent': 'None'}

In [10]:
import random
random.shuffle(subset)

In [11]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
from tqdm import tqdm

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
call_count=0
def extract_wav2vec_features(batch):
  global call_count
  call_count+=1
  print(call_count)
  inputs = processor(batch["audio"]["array"],sampling_rate=16000,return_tensors='pt',padding=True)
  with torch.no_grad():
    outputs = model(**inputs)
  return {"features": outputs.last_hidden_state.squeeze().mean(dim=0).numpy(),"label":batch["language"]}

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
subset_with_features = [extract_wav2vec_features(example) for example in subset]

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152


In [14]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(subset_with_features, test_size=0.2)
train_data, val_data = train_test_split(train_data,test_size=0.1)

def prepare_data(data):
  features = [item["features"] for item in data]
  labels = [item["label"] for item in data]
  return torch.tensor(features),torch.tensor(labels)

train_features,train_labels = prepare_data(train_data)
val_features, val_labels = prepare_data(val_data)
test_features, test_labels = prepare_data(test_data)

<ipython-input-14-f58d082f57da>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(features),torch.tensor(labels)


In [23]:
train_labels.unique(),val_labels.unique(),test_labels.unique()

(tensor([0, 3, 6]), tensor([0, 3, 6]), tensor([0, 3, 6]))

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNNModel(nn.Module):
  def __init__(self, input_dim, num_classes):
    super(CNNModel, self).__init__()
    self.conv1 = nn.Conv1d(in_channels =input_dim,out_channels=128, kernel_size=3,padding=1)
    self.conv2 = nn.Conv1d(in_channels =128, out_channels=256, kernel_size=3, padding=1)
    self.fc1= nn.Linear(256*(input_length//4),128)
    self.fc2= nn.Linear(128, num_classes)

  def forward(self,x):
    x = F.relu(self.conv1(x))
    x = F.max_pool1d(x,kernel_size=2)
    x = F.relu(self.conv2(x))
    x = F.max_pool1d(x,kernel_size=2)
    x = x.view(x.size(0),-1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [24]:
input_length = train_features.shape[1]
label_mapping = {6: 0, 0: 1, 3: 2}

# label_mapping = {6: 0, 0: 1, 3: 2, 1: 3, 2: 4}  # Adjust mappings as per your needs


train_labels = train_labels.apply_(lambda x: label_mapping[x])
test_labels = test_labels.apply_(lambda x: label_mapping[x])
val_labels = val_labels.apply_(lambda x: label_mapping[x])

unique_train_labels = torch.unique(train_labels)
unique_test_labels = torch.unique(test_labels)
unique_val_labels = torch.unique(val_labels)

In [25]:
print(torch.unique(train_labels))
print(torch.unique(test_labels))
print(torch.unique(val_labels))


tensor([0, 1, 2])
tensor([0, 1, 2])
tensor([0, 1, 2])


In [29]:
model = CNNModel(input_dim=1,num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)


In [32]:
num_epochs = 20
for epoch in range(num_epochs):
  model.train()
  optimizer.zero_grad()
  output = model(train_features.unsqueeze(1).float())
  loss = criterion(output,train_labels)
  loss.backward()
  optimizer.step()

  #validation
  model.eval()
  with torch.no_grad():
    val_outputs = model(val_features.unsqueeze(1).float())
    val_loss = criterion(val_outputs,val_labels)

  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()},val Loss: {val_loss.item()}")

Epoch 1/20, Loss: 1.099430799484253,val Loss: 5.919916152954102
Epoch 2/20, Loss: 3.783783435821533,val Loss: 1.3603636026382446
Epoch 3/20, Loss: 1.1326133012771606,val Loss: 0.9851166605949402
Epoch 4/20, Loss: 1.1436671018600464,val Loss: 0.9460722804069519
Epoch 5/20, Loss: 1.1086134910583496,val Loss: 0.9682071208953857
Epoch 6/20, Loss: 1.0916509628295898,val Loss: 0.9876452088356018
Epoch 7/20, Loss: 1.062320351600647,val Loss: 1.0044957399368286
Epoch 8/20, Loss: 1.0365393161773682,val Loss: 1.033638596534729
Epoch 9/20, Loss: 1.0170071125030518,val Loss: 1.0598472356796265
Epoch 10/20, Loss: 0.9870029091835022,val Loss: 1.045922040939331
Epoch 11/20, Loss: 0.9522685408592224,val Loss: 0.9956141114234924
Epoch 12/20, Loss: 0.9052849411964417,val Loss: 0.9090936779975891
Epoch 13/20, Loss: 0.8667086362838745,val Loss: 0.7870375514030457
Epoch 14/20, Loss: 0.8114835023880005,val Loss: 0.7136807441711426
Epoch 15/20, Loss: 0.7552503943443298,val Loss: 0.6615952849388123
Epoch 16/2

In [34]:
import torch
from sklearn.metrics import accuracy_score

model.eval()
with torch.no_grad():
  test_outputs = model(test_features.unsqueeze(1).float())
  _, predicted = torch.max(test_outputs,1)

  accuracy = accuracy_score(test_labels.numpy(),predicted.numpy())

print(f"Test Accuracy: {accuracy *100:.2f}%")

Test Accuracy: 66.67%
